<a href="https://colab.research.google.com/github/sebastianfjrd/Project-3_Group-9/blob/main/8_18_prototype_Gradio_Image_upload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio
!pip install google-cloud-storage
!pip install gradio google-cloud-storage googlemaps
!pip install tensorflow
!pip install geopy
!pip install gradio SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 322.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing 

In [2]:
import gradio as gr
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model # To load the model later to make predictions
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import speech_recognition as sr
import requests

In [3]:
# Load the saved model
def load_melanoma_model():
    try:
        model_path = r'C:\Users\sebas\OneDrive\Desktop\_CU-VIRT-AI-PT-03-2024-U-LOLC\ClassProject\Project 3\melanoma_cnn_model.keras'
        model = tf.keras.models.load_model(model_path)
        print("Model loaded successfully!")
        return model
    except Exception as e:
        print(f"Error loading the model: {e}")
        return None

In [4]:
# Format image for prediction
def format_image_for_prediction(image):
    image = tf.image.resize(image, (300, 300))
    image_normalized = np.array(image).astype(np.float32) / 255
    image_normalized = np.expand_dims(image_normalized, axis=0)  # Shape becomes (1, 300, 300, 3)
    return image_normalized

In [5]:
# Get prediction from the model
def get_prediction(image_normalized):
    model = load_melanoma_model()
    if model is None:
        return "Error loading the model. Please try again later."

    try:
        prediction_probs = model.predict(image_normalized)
        prediction = np.argmax(prediction_probs, axis=1)
        confidence = np.max(prediction_probs)
        if prediction == 1:
            result = f"Our model's prediction is malignant with {confidence*100:.2f}% confidence."
        else:
            result = f"Our model's prediction is benign with {confidence*100:.2f}% confidence."
        return result
    except Exception as e:
        return f"Error during prediction: {e}"

In [6]:
# Process image and get prediction
def process_image(image):
    image_normalized = format_image_for_prediction(image)
    prediction = get_prediction(image_normalized)
    return prediction

In [7]:
# Find cancer specialists using NPI Registry API (Example)
def find_specialists(zip_code):
    try:
        api_url = f"https://npiregistry.cms.hhs.gov/api/?version=2.1&number=&enumeration_type=NPI-1&taxonomy_description=Oncology&postal_code={zip_code}&country_code=US&limit=5"
        response = requests.get(api_url)
        if response.status_code == 200:
            data = response.json()
            specialists = data.get('results', [])
            if specialists:
                results = "\n".join([
                    f"{spec.get('basic', {}).get('name', 'Name not found')}, "
                    f"{spec.get('addresses', [{}])[0].get('address_1', 'Address not found')}, "
                    f"{spec.get('addresses', [{}])[0].get('city', 'City not found')}, "
                    f"{spec.get('addresses', [{}])[0].get('state', 'State not found')}"
                    for spec in specialists
                ])
                return f"Top cancer specialists near {zip_code}:\n{results}"
            else:
                return f"No cancer specialists found near {zip_code}."
        else:
            return f"Error retrieving data: {response.status_code}"
    except Exception as e:
        return f"Error finding specialists: {e}"


In [8]:
# Speech-to-text function
def speech_to_text(audio):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
            return text
        except sr.UnknownValueError:
            return "Sorry, I did not understand the audio."
        except sr.RequestError as e:
            return f"Could not request results; {e}"

In [9]:
# Gradio app interface
def melanoma_scan_app(image, zip_code, doctor_notes_audio):
    prediction = process_image(image)
    specialists = find_specialists(zip_code)
    doctor_notes_text = speech_to_text(doctor_notes_audio)
    return f"{prediction}\n\n{specialists}\n\nDoctor's Notes: {doctor_notes_text}"

# User instructions
def user_instructions():
    return ("Instructions:\n"
            "1. Upload an image of a skin lesion.\n"
            "2. Enter your zip code to find nearby cancer specialists if needed.\n"
            "3. Record a doctor's spoken notes if available.\n"
            "4. Click 'Submit' to see the model's prediction, nearby specialists, and any additional notes.\n"
            "Note: This tool is for informational purposes only and not a substitute for professional medical advice.")

In [10]:
# Creating Gradio interface
interface = gr.Interface(
    fn=melanoma_scan_app,
    inputs=[
        gr.Image(type="pil", label="Upload a Skin Lesion Image"),
        gr.Textbox(label="Enter Your Zip Code"),
        gr.Audio(type="filepath", label="Doctor's Notes")  # Corrected 'type' to "filepath"
    ],
    outputs="text",
    title="Melanoma Detection & Specialist Finder",
    description=user_instructions(),
    allow_flagging="never"
)

In [11]:
import requests

def find_specialists(zip_code):
    try:
        # Hypothetical API URL
        api_url = f"https://api.healthcare.com/specialists?zip={zip_code}&specialty=cancer"

        # Send a request to the API
        response = requests.get(api_url)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            if data:
                results = "\n".join([f"{spec['name']}, {spec['address']}" for spec in data[:5]])
                return f"Top cancer specialists near {zip_code}:\n{results}"
            else:
                return f"No cancer specialists found near {zip_code}."
        else:
            return f"Error retrieving data: {response.status_code}"
    except Exception as e:
        return f"Error finding specialists: {e}"

In [12]:
import requests

def find_specialists(zip_code):
    try:
        # NPI Registry API URL with parameters for specialty and location
        api_url = f"https://npiregistry.cms.hhs.gov/api/?version=2.1&number=&enumeration_type=NPI-1&taxonomy_description=Oncology&postal_code={zip_code}&country_code=US&limit=5"

        # Send a request to the NPI Registry API
        response = requests.get(api_url)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            specialists = data.get('results', [])
            if specialists:
                results = "\n".join([f"{spec['basic']['name']}, {spec['addresses'][0]['address_1']}, {spec['addresses'][0]['city']}, {spec['addresses'][0]['state']}" for spec in specialists])
                return f"Top cancer specialists near {zip_code}:\n{results}"
            else:
                return f"No cancer specialists found near {zip_code}."
        else:
            return f"Error retrieving data: {response.status_code}"
    except Exception as e:
        return f"Error finding specialists: {e}"

In [13]:
import requests

def find_specialists(zip_code):
    try:
        # Example: Using NPI Registry API (You can replace this with the actual API you're using)
        api_url = f"https://npiregistry.cms.hhs.gov/api/?version=2.1&number=&enumeration_type=NPI-1&taxonomy_description=Oncology&postal_code={zip_code}&country_code=US&limit=5"

        # Send a request to the API
        response = requests.get(api_url)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            print("API Response:", data)  # Debug print to see the full API response

            specialists = data.get('results', [])
            if specialists:
                for spec in specialists:
                    # Print out the entire specialist data for each entry
                    print("Specialist Data (Full):", spec)

                # Still attempting to access name based on previous assumptions
                results = "\n".join([
                    f"{spec.get('basic', {}).get('name', 'Name not found')}, "
                    f"{spec.get('addresses', [{}])[0].get('address_1', 'Address not found')}, "
                    f"{spec.get('addresses', [{}])[0].get('city', 'City not found')}, "
                    f"{spec.get('addresses', [{}])[0].get('state', 'State not found')}"
                    for spec in specialists
                ])
                return f"Top cancer specialists near {zip_code}:\n{results}"
            else:
                return f"No cancer specialists found near {zip_code}."
        else:
            return f"Error retrieving data: {response.status_code}"
    except Exception as e:
        return f"Error finding specialists: {e}"




In [14]:
# Launch the app
interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2f78218390ede13fe7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
